In [1]:
!pip install onnxruntime
import onnxruntime as ort
from keras import utils
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, classification_report


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


2024-08-28 08:41:24.177500: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 08:41:24.177560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 08:41:24.179085: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 08:41:24.187404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 08:41:25.138248: W tensorflow/comp

In [2]:
path_to_data = 'validate/'

In [3]:
val_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    shuffle=True,
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

Found 30 files belonging to 10 classes.


In [4]:
for images,lables in val_data.take(1):
  print('Amount of train images: ', images.shape)
  print('Amount of train labels: ', lables.shape)

Amount of train images:  (30, 256, 256, 3)
Amount of train labels:  (30,)


In [5]:
def normalize(image, label):
  return image/255.0, label

In [6]:
val_data= val_data.map(normalize)

In [7]:
val_x=[]
val_y=[]

for image,label in val_data:
   val_x.append(image)
   val_y.append(label)

val_x = tf.concat(val_x, axis=0)
val_y = tf.concat(val_y, axis=0)

In [8]:
num_classes = 10
val_y = tf.keras.utils.to_categorical(val_y, num_classes=num_classes)

In [9]:
new_model = 'model-5-epochs.onnx'
new_model_session = ort.InferenceSession(new_model)

production_model = 'model-4-epochs.onnx'
production_model_session = ort.InferenceSession(production_model)

new_model_input = new_model_session.get_inputs()[0].name
new_model_output = new_model_session.get_outputs()[0].name

production_model_input = production_model_session.get_inputs()[0].name
production_model_output = production_model_session.get_outputs()[0].name

In [10]:
new_pred_arr = []
production_pred_arr = []

for i, val in enumerate(val_x):
    new_model_pred = new_model_session.run([new_model_output], {new_model_input: [val_x[i].cpu().numpy()]})
    production_model_pred = production_model_session.run([production_model_output], {production_model_input: [val_x[i].cpu().numpy()]})

    new_pred_arr.append(new_model_pred)
    production_pred_arr.append(production_model_pred)

Instructions for updating:
Use tf.identity with explicit device placement instead.


In [11]:
new_pred_arr = np.concatenate([np.argmax(pred[0], axis=1) for pred in new_pred_arr], axis=0)
production_pred_arr = np.concatenate([np.argmax(pred[0], axis=1) for pred in production_pred_arr], axis=0)

val_y_flat = np.argmax(val_y, axis=1)

new_model_accuracy = accuracy_score(val_y_flat, new_pred_arr)
production_model_accuracy = accuracy_score(val_y_flat, production_pred_arr)

print(f"New Model Accuracy: {new_model_accuracy * 100:.2f}%")
print(f"Production Model Accuracy: {production_model_accuracy * 100:.2f}%")

print("\nNew Model Classification Report:")
print(classification_report(val_y_flat, new_pred_arr, zero_division=0))

print("\nProduction Model Classification Report:")
print(classification_report(val_y_flat, production_pred_arr, zero_division=0))

New Model Accuracy: 26.67%
Production Model Accuracy: 43.33%

New Model Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         3
           1       0.50      0.67      0.57         3
           2       0.20      0.67      0.31         3
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         3
           5       1.00      0.33      0.50         3
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         3
           8       0.14      0.33      0.20         3
           9       1.00      0.33      0.50         3

    accuracy                           0.27        30
   macro avg       0.38      0.27      0.26        30
weighted avg       0.38      0.27      0.26        30


Production Model Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.33      0.40       

In [ ]:
should_replace = new_model_accuracy > production_model_accuracy

with open('should_replace.json', 'w') as f:
    json.dump({'replace': should_replace}, f)

In [ ]:
import os
import zipfile

with zipfile.ZipFile('replacement.zip', 'w') as zipf:
    zipf.write('should_replace.json')
    if should_replace:
        zipf.write('new_model.onnx')